In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from torchtext.vocab import vocab

# Define the tokenizer
def char_tokenizer(text):
    return list(text)

# Load dataset (Example placeholder, replace with actual data loading code)
train_dataset = [("This is an example sentence.", 0), ("Another sentence for training.", 1)]
counter = Counter()
for (line, label) in train_dataset:
    counter.update(char_tokenizer(line))
vocab_obj = vocab(counter)

# Define LSTM-based generative network
class LSTMGenerator(nn.Module):
    def __init__(self, vocab_size, hidden_dim, n_layers=1):
        super(LSTMGenerator, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = nn.LSTM(vocab_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

# Helper function to encode text
def one_hot_encode(text, vocab_obj):
    tensor = torch.zeros(len(text), len(vocab_obj))
    for i, char in enumerate(text):
        tensor[i][vocab_obj[char]] = 1
    return tensor

# Define training parameters
vocab_size = len(vocab_obj)
hidden_dim = 128
n_layers = 2
n_epochs = 2000
lr = 0.001

# Initialize model, loss function, and optimizer
model = LSTMGenerator(vocab_size, hidden_dim, n_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training loop
for epoch in range(n_epochs):
    for (line, label) in train_dataset:
        hidden = model.init_hidden(1)
        model.zero_grad()
        input_seq = one_hot_encode(line[:-1], vocab_obj).unsqueeze(0)
        target_seq = torch.tensor([vocab_obj[char] for char in line[1:]])
        
        output, hidden = model(input_seq, hidden)
        loss = criterion(output.squeeze(), target_seq)
        loss.backward()
        optimizer.step()
    
    if epoch % 100 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.item()}')

# Function to generate text
def generate_text(model, start_text, length=100):
    model.eval()
    hidden = model.init_hidden(1)
    input_seq = one_hot_encode(start_text, vocab_obj).unsqueeze(0)
    chars = [ch for ch in start_text]
    
    with torch.no_grad():
        for _ in range(length):
            output, hidden = model(input_seq, hidden)
            topv, topi = output.topk(1)
            char_index = topi[0, -1].item()
            char = vocab_obj.lookup_token(char_index)
            chars.append(char)
            input_seq = one_hot_encode(chars[-1], vocab_obj).unsqueeze(0)
    
    return ''.join(chars)

# Generate and print text
print(generate_text(model, start_text="This is"))


Epoch: 0, Loss: 2.9966180324554443
Epoch: 100, Loss: 0.840171217918396
Epoch: 200, Loss: 0.28384754061698914
Epoch: 300, Loss: 0.07948210090398788
Epoch: 400, Loss: 0.032600242644548416
Epoch: 500, Loss: 0.017976846545934677
Epoch: 600, Loss: 0.011216194368898869
Epoch: 700, Loss: 0.00785580463707447
Epoch: 800, Loss: 0.005819302052259445
Epoch: 900, Loss: 0.004371980205178261
Epoch: 1000, Loss: 0.0034466085489839315
Epoch: 1100, Loss: 0.0027934431564062834
Epoch: 1200, Loss: 0.0022935373708605766
Epoch: 1300, Loss: 0.0019028299720957875
Epoch: 1400, Loss: 0.0015955442795529962
Epoch: 1500, Loss: 0.001346526900306344
Epoch: 1600, Loss: 0.001144013600423932
Epoch: 1700, Loss: 0.0009799095569178462
Epoch: 1800, Loss: 0.0008454998023808002
Epoch: 1900, Loss: 0.0007344027399085462
This is an example sentence..oo trrinnnnc...noh  ssntence for training...nooh  sentence for training...noot
